In [1]:
import pandas as pd
data = pd.read_csv('cleaned_data.csv')

In [2]:
data.head(3)

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN


In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
#can't get it to work will use lemmatizer from class
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#prepare stopwords
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]


In [5]:
#make df for only rock genre
rockdf = data.loc[data['Genre1'] =='Rock']
rockdf

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
14,15,Atardecer,Friends of Dean Martinez,the moment i saw the packaging i knew somethin...,"Knitting Factory,Knitting Factory",8.3,March,23,1999,Rock,NaN,NaN,NaN
20,21,Seven More Minutes,The Rentals,"""i don't think this relationship is going anyw...","Reprise,Maverick,Reprise,Maverick",5.9,April,13,1999,Rock,NaN,NaN,NaN
25,26,The Sound of Music: An Unfinished Symphony in ...,Jad Fair,"many years ago, i picked up bongwater's ,. i k...","Shimmy Disc,Shimmy Disc",3.7,April,20,1999,Rock,NaN,NaN,NaN
27,28,The Hush,Texas,england's texas sculpts studio-perfect summer ...,"Interscope,Interscope",4.0,May,18,1999,Rock,NaN,NaN,NaN
29,30,Agaetis Byrjun,Sigur Rós,icelandic lore tells of the hidden people who ...,Smekkleysa,9.4,June,1,1999,Rock,NaN,NaN,NaN
32,33,The Only Reason I Feel Secure EP,Pedro the Lion,i was going through the magazine rack at local...,"Made in Mexico,Made in Mexico",8.4,June,1,1999,Rock,NaN,NaN,NaN
37,38,EEVIAC,Man or Astro-Man?,"good evening, earthlings. the dreaded planet s...","Touch and Go,Touch and Go",5.9,April,20,1999,Rock,NaN,NaN,NaN


In [6]:
#make df for only rock in 2018
rockdf18 = rockdf.loc[rockdf['Year']== '2018']
rockdf18

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
76,77,A Brief Inquiry Into Online Relationships,The 1975,dare to be too much. led by frontman and lyri...,"Dirty Hit,Interscope",8.5,November,29,2018,Rock,NaN,NaN,NaN
80,81,“Ouch”,Nicholas Krgovich,“you learn that so much of the fear and anxiet...,Tin Angel,7.2,November,29,2018,Rock,NaN,NaN,NaN
82,83,Look Now Look Again,Rainer Maria,met as 21-year-olds in a college poetry class...,Polyvinyl,8.3,November,29,2018,Rock,NaN,NaN,NaN
83,84,Anthem of the Peaceful Army,Greta Van Fleet,greta van fleet sound like they did weed exact...,"Lava,Republic",1.6,October,23,2018,Rock,NaN,NaN,NaN
85,86,The Bela Session,Bauhaus,everyone knows “bela lugosi’s dead.” for decad...,Leaving,7.9,December,1,2018,Rock,NaN,NaN,NaN
93,94,Songs of Love and Horror,Will Oldham,"near the end of a recent show in brooklyn, , b...",Drag City,7.1,October,24,2018,Rock,NaN,NaN,NaN
95,96,Bohemian Rhapsody OST,Queen,"have now existed longer as a band , freddie m...",Virgin,4.5,October,25,2018,Rock,NaN,NaN,NaN


In [7]:
#tokenize words and clean text 
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(rockdf18['content']))

print(data_words[:1])

[['you', 'know', 'how', 'life', 'goes', 'one', 'day', 'you', 're', 'making', 'songs', 'like', 'selling', 'with', 'vagina', 'bat', 'flying', 'above', 'your', 'band', 'name', 'and', 'years', 'go', 'by', 'and', 'suddenly', 'everyone', 'getting', 'married', 'and', 'having', 'kids', 'after', 'all', 'the', 'actual', 'blood', 'piss', 'and', 'sweat', 'that', 'went', 'into', 'mocking', 'indie', 'culture', 'and', 'convention', 'your', 'friends', 'have', 'somehow', 'become', 'craft', 'brewers', 'and', 'commercial', 'music', 'writers', 'rhode', 'island', 'hardcore', 'misanthropes', 'never', 'seemed', 'the', 'sort', 'of', 'band', 'compatible', 'with', 'the', 'demands', 'of', 'adulthood', 'when', 'they', 'broke', 'up', 'in', 'due', 'to', 'extreme', 'internal', 'duress', 'there', 'was', 'little', 'reason', 'to', 'believe', 'some', 'newfound', 'cachet', 'changing', 'times', 'or', 'popular', 'demand', 'would', 'ever', 'coax', 'their', 'wildly', 'abrasive', 'music', 'back', 'into', 'action', 'they', 'ac

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['you', 'know', 'how', 'life', 'goes', 'one', 'day', 'you', 're', 'making', 'songs', 'like', 'selling', 'with', 'vagina', 'bat', 'flying', 'above', 'your', 'band', 'name', 'and', 'years', 'go', 'by', 'and', 'suddenly', 'everyone', 'getting', 'married', 'and', 'having', 'kids', 'after', 'all', 'the', 'actual', 'blood', 'piss', 'and', 'sweat', 'that', 'went', 'into', 'mocking', 'indie', 'culture', 'and', 'convention', 'your', 'friends', 'have', 'somehow', 'become', 'craft', 'brewers', 'and', 'commercial', 'music', 'writers', 'rhode', 'island', 'hardcore', 'misanthropes', 'never', 'seemed', 'the', 'sort', 'of', 'band', 'compatible', 'with', 'the', 'demands', 'of', 'adulthood', 'when', 'they', 'broke', 'up', 'in', 'due', 'to', 'extreme', 'internal', 'duress', 'there', 'was', 'little', 'reason', 'to', 'believe', 'some', 'newfound', 'cachet', 'changing', 'times', 'or', 'popular', 'demand', 'would', 'ever', 'coax', 'their', 'wildly', 'abrasive', 'music', 'back', 'into', 'action', 'they', 'act

In [9]:
print(bigram)

Phrases<191899 vocab, min_count=5, threshold=100, max_vocab_size=40000000>


In [10]:
## Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = data_words_bigrams

print(data_lemmatized[:1])

[['know', 'life', 'goes', 'one', 'day', 'making', 'songs', 'like', 'selling', 'vagina', 'bat', 'flying', 'band', 'name', 'years', 'go', 'suddenly', 'everyone', 'getting', 'married', 'kids', 'actual', 'blood', 'piss', 'sweat', 'went', 'mocking', 'indie', 'culture', 'convention', 'friends', 'somehow', 'become', 'craft', 'brewers', 'commercial', 'music', 'writers', 'rhode', 'island', 'hardcore', 'misanthropes', 'never', 'seemed', 'sort', 'band', 'compatible', 'demands', 'adulthood', 'broke', 'due', 'extreme', 'internal', 'duress', 'little', 'reason', 'believe', 'newfound', 'cachet', 'changing', 'times', 'popular', 'demand', 'would', 'ever', 'coax', 'wildly', 'abrasive', 'music', 'back', 'action', 'actually', 'tricked', 'reuniting', 'anyway', 'organizing', 'principles', 'plans', 'besides', 'lingering', 'contempt', 'humanity', 'say', 'way', 'title', 'fourth', 'album', 'first', 'eight', 'years', 'really', 'mean', 'get', 'deserve', 'daughters', 'past', 'work', 'provided', 'instantaneous', 'ab

In [16]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 2), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 5), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 2), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 2), (110, 1)

In [17]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  scor

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  scor

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  scor

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  scor

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  scor

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  scor

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  scor

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  scor

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  scor

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  scor

In [18]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.007*"vile" + 0.005*"mogwai" + 0.003*"barnes" + 0.003*"post_animal" + '
  '0.003*"bassackwards" + 0.002*"film" + 0.002*"krgovich" + 0.002*"eli" + '
  '0.002*"tweet" + 0.002*"predilection"'),
 (1,
  '0.012*"dylan" + 0.004*"morrison" + 0.004*"brotherhood" + 0.003*"barnett" + '
  '0.003*"murdoch" + 0.003*"jeff" + 0.003*"robertson" + 0.003*"helm" + '
  '0.003*"women" + 0.003*"marshall"'),
 (2,
  '0.006*"bowie" + 0.004*"pop" + 0.003*"band" + 0.003*"jones" + 0.003*"song" + '
  '0.003*"omori" + 0.003*"strummer" + 0.003*"white" + 0.002*"harvey" + '
  '0.002*"last"'),
 (3,
  '0.007*"like" + 0.004*"album" + 0.003*"waits" + 0.003*"songs" + '
  '0.003*"record" + 0.003*"sings" + 0.002*"voice" + 0.002*"one" + 0.002*"time" '
  '+ 0.002*"song"'),
 (4,
  '0.006*"dunn" + 0.006*"costello" + 0.005*"kenney" + 0.005*"mudhoney" + '
  '0.005*"fraser" + 0.005*"kirke" + 0.004*"dodos" + 0.004*"lambchop" + '
  '0.004*"third_eye" + 0.004*"oh_sees"'),
 (5,
  '0.012*"bikini_kill" + 0.009*"hanna" + 0.008*"sh

In [19]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  scor

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  scor

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  scor


Perplexity:  -9.058152498001881


/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  scor


Coherence Score:  0.5218790816112023


In [20]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '/Users/davidbrostowitz/Downloads/mallet-2.0.8' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

CalledProcessError: Command '/Users/davidbrostowitz/Downloads/mallet-2.0.8 import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /var/folders/mk/7qm4p1js4hl0ppgjz0s_wwdh0000gn/T/b961f8_corpus.txt --output /var/folders/mk/7qm4p1js4hl0ppgjz0s_wwdh0000gn/T/b961f8_corpus.mallet' returned non-zero exit status 126.